# DNA Melting
A suite of several machine learning models to predict DNA melting temperature. For all of intents and purposes of this notebook, we only need to know that this is a number that depends on two other numbers (the duplex and salt concentrations) and on a string. Each string represents a DNA sequence, hence it is composed of A, C, G, T. Similar strings tend to have a similar melting temperature.

First, let us import a few packages

In [67]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import sklearn
import SantaLucia as sl
import make_melting_database
import glob, os
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import TensorDataset, DataLoader

bases = ['A', 'C', 'G', 'T']

Only for the first time that the notebook is run, or when we want to add/change the data stored in the melting temperature database, we use `write_database` from `make_melting_database.py` to generate the melting temperature database.

In [68]:
write_database = False
if write_database:
    start_n_bases = 6
    end_n_bases = 9
    duplex_umol_conc = [1,2]
    salt_mol_conc = [0.1, 0.2, 0.5]
    make_melting_database.write_database(start_n_bases, end_n_bases, duplex_umol_conc, salt_mol_conc)

Then, we can load the dataset as a dataframe

In [69]:
# gather the data from the various files and pool them into one dataframe
database_dir = 'melting_database'
datasets = os.listdir(database_dir)

all_files = [os.path.join(database_dir, data) for data in datasets]
print(all_files)
df = pd.concat((pd.read_csv(f, index_col=0) for f in all_files), ignore_index=True)

df
#dataset = torch.utils.data.TensorDataset(x, T_m)
#dataloader = torch.utils.data.DataLoader(dataset)

['melting_database\\6meres.csv', 'melting_database\\7meres.csv', 'melting_database\\8meres.csv', 'melting_database\\9meres.csv']


,seq,s_mol,c_umol,T_m,base_0,base_1,base_2,base_3,base_4,base_5,base_6,base_7,base_8
0,AAAAAA,0.1,1,-20.495855,A,A,A,A,A,A,0,0,0
1,AAAAAA,0.2,1,-17.536560,A,A,A,A,A,A,0,0,0
2,AAAAAA,0.5,1,-13.516521,A,A,A,A,A,A,0,0,0
3,AAAAAA,0.1,2,-17.837395,A,A,A,A,A,A,0,0,0
4,AAAAAA,0.2,2,-14.815124,A,A,A,A,A,A,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2088955,GGGGGGGGG,0.2,1,47.545073,G,G,G,G,G,G,G,G,G
2088956,GGGGGGGGG,0.5,1,52.512866,G,G,G,G,G,G,G,G,G
2088957,GGGGGGGGG,0.1,2,46.070145,G,G,G,G,G,G,G,G,G
2088958,GGGGGGGGG,0.2,2,49.779454,G,G,G,G,G,G,G,G,G


## 1. One-hot encoded bases fed to a fully connected network

The simplest model we will try is a fully connected network attached connected to one-hot encoded nucleotides. First of all, we perform the one-hot encoding

In [70]:
df_oh = df.copy()
# We add (and then remove) a sequence with all missing bases to ensure that
# every position has the same number of dummy columns. Not required for now,
# but required for steps 3 and 4
df_oh.loc[df.shape[0]] = df.shape[1] * [0]

i = 0
while True:
    target_col = 'base_' + str(i)
    # get out of the loop when we reach the last 'base_n' column
    try:
        df[target_col]
    except KeyError:
        break
    # generate the one-hot encoded columns and append them to the dataframe
    new_cols = pd.get_dummies(df_oh[target_col], prefix = target_col)
    df_oh = pd.concat([df_oh, new_cols], axis=1)
    # remove original column from the dataframe
    df_oh.drop(target_col, axis=1, inplace=True)
    i += 1
df_oh.drop('seq', axis=1, inplace=True)
df_oh.drop(df_oh.shape[0]-1,inplace=True)
df_oh

,s_mol,c_umol,T_m,base_0_0,base_0_A,base_0_C,base_0_G,base_0_T,base_1_0,base_1_A,...,base_7_0,base_7_A,base_7_C,base_7_G,base_7_T,base_8_0,base_8_A,base_8_C,base_8_G,base_8_T
0,0.1,1,-20.495855,0,1,0,0,0,0,1,...,1,0,0,0,0,1,0,0,0,0
1,0.2,1,-17.536560,0,1,0,0,0,0,1,...,1,0,0,0,0,1,0,0,0,0
2,0.5,1,-13.516521,0,1,0,0,0,0,1,...,1,0,0,0,0,1,0,0,0,0
3,0.1,2,-17.837395,0,1,0,0,0,0,1,...,1,0,0,0,0,1,0,0,0,0
4,0.2,2,-14.815124,0,1,0,0,0,0,1,...,1,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2088955,0.2,1,47.545073,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2088956,0.5,1,52.512866,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2088957,0.1,2,46.070145,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2088958,0.2,2,49.779454,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0


Then we split the data between the training set and the validation set, and load it into dataloaders

In [71]:
# Cast the data into a dataset
y = torch.Tensor(df.T_m, device='cuda')
x = torch.Tensor(df_oh.drop('T_m', axis=1).values, device='cuda')


dataset = TensorDataset(x, y)

train_length = int(len(dataset) * 0.9)
valid_length = len(dataset) - train_length
train_set, valid_set = torch.utils.data.random_split(dataset, [train_length, valid_length], generator=torch.Generator(device='cuda'))

# Report split sizes
print(f'Training set has {len(train_set)} instances')
print(f'Validation set has {len(valid_set)} instances')

# Create the loaders
batch_size = 64
training_loader = DataLoader(train_set, batch_size=batch_size, shuffle=False, num_workers=2)
validation_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False, num_workers=2)

Training set has 1880064 instances
Validation set has 208896 instances


Now let's define the model. In this part of the notebook we use a simple fully connected network.

In [72]:
from turtle import forward

class FC(nn.Module):
    def __init__(self, input_size : int, n_hidden_layers : int, hidden_size : int):
        super(FC, self).__init__()
        self.n_hidden_layers = n_hidden_layers

        self.input_layer = nn.Linear(input_size, hidden_size)
        self.hidden_layer = nn.Linear(hidden_size, hidden_size)
        self.dropout = nn.Dropout()
        self.output_layer = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        x = F.relu(self.input_layer(x))
        for n in range(self.n_hidden_layers):
            x = F.relu(self.hidden_layer(x))
            x = self.dropout(x)
        x = self.output_layer(x)
        return x

model = FC(x.shape[1], 1, x.shape[1] * 3).cuda()

We carry on defining the loss and the optimizer. We use a standard MSE loss and an Adam optimizer with default parameters.

In [73]:
#loss_fn = nn.MSELoss()
loss_fn = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters())

Finally, the training loop - taken from https://pytorch.org/tutorials/beginner/introyt/trainingyt.html.

In [74]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(training_loader):
        # Every data instance is an input + label pair
        inputs, labels = data

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        labels = torch.unsqueeze(labels, 1)
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(training_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss

The training loop sweeps the data over multiple epochs. Once per epoch, we perform model validation by checking model performance on validation data, and save a copy of the model

In [75]:
# Initializing in a separate cell so we can easily add more epochs to the same run
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))
epoch_number = 0

EPOCHS = 5

best_vloss = 1_000_000.

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch(epoch_number, writer)

    # We don't need gradients on to do reporting
    model.train(False)

    running_vloss = 0.0
    for i, vdata in enumerate(validation_loader):
        vinputs, vlabels = vdata
        voutputs = model(vinputs)
        vlabels = torch.unsqueeze(vlabels, 1) # added to ensure labels and outputs are of the same size
        vloss = loss_fn(voutputs, vlabels)
        running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    # Log the running loss averaged per batch
    # for both training and validation
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch_number + 1)
    writer.flush()

    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = 'model_{}_{}'.format(timestamp, epoch_number)
        torch.save(model.state_dict(), model_path)

    epoch_number += 1

EPOCH 1:


AttributeError: module 'torch.cuda' has no attribute '_UntypedStorage'

When training the model to reproducing melting temperatures of sequences with length between 6 and 9, the MSE loss plateaus at 6.4, while the L1 loss gets to 1.7. Not bad, given the wide range of melting temperatures observed.

In [260]:
inputs, labels = next(iter(training_loader))
#labels
x = 
print(model(inputs)[:inputs.cpu().numpy()[0,:].shape[0],0].detach().numpy().flatten().shape)
print(inputs.cpu().numpy()[0,:].shape)

df_plot = pd.DataFrame({'predicted' : model(inputs)[:,0].detach().numpy()[:inputs.cpu().numpy().shape[0]], 'computed' : inputs.cpu().numpy()[0]})
#df_plot.plot()

(41,)
(41,)


ValueError: All arrays must be of the same length

In [230]:
for i, data in enumerate(training_loader):
    inputs, labels = data

    outputs = model(inputs)

    loss = loss_fn(outputs, labels)
    break
for i, vdata in enumerate(validation_loader):
        vinputs, vlabels = vdata
        voutputs = model(vinputs)
        #vlabels = torch.unsqueeze(vlabels, 1)
        vlabels = torch.unsqueeze(vlabels, 1)
        vloss = loss_fn(voutputs, vlabels)
        break

## 2. One-hot encoded base-pair steps fed to a fully connected network
First of all, we need to generate the columns containing the base-pair step of each sequence


In [30]:
df_bps = df.copy()
i = 0
while True:
    base = 'base_'
    # get out of the loop when we reach the last 'base_n' column
    try:
        df[base + str(i + 1)]
    except KeyError:
        break
    # concatenate two columns to make base-pair steps. The 'astype(str)' are necessary to deal with the number-like 0 values in df_bps.
    df_bps[base + str(i) + str(i + 1)] = df[base + str(i)].astype(str) + df[base + str(i+1)].astype(str)
    i += 1    
df_bps

,seq,s_mol,c_umol,T_m,base_0,base_1,base_2,base_3,base_4,base_5,...,base_7,base_8,base_01,base_12,base_23,base_34,base_45,base_56,base_67,base_78
0,AAAAAA,0.1,1,-20.495855,A,A,A,A,A,A,...,0,0,AA,AA,AA,AA,AA,A0,00,00
1,AAAAAA,0.2,1,-17.536560,A,A,A,A,A,A,...,0,0,AA,AA,AA,AA,AA,A0,00,00
2,AAAAAA,0.5,1,-13.516521,A,A,A,A,A,A,...,0,0,AA,AA,AA,AA,AA,A0,00,00
3,AAAAAA,0.1,2,-17.837395,A,A,A,A,A,A,...,0,0,AA,AA,AA,AA,AA,A0,00,00
4,AAAAAA,0.2,2,-14.815124,A,A,A,A,A,A,...,0,0,AA,AA,AA,AA,AA,A0,00,00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2088955,GGGGGGGGG,0.2,1,47.545073,G,G,G,G,G,G,...,G,G,GG,GG,GG,GG,GG,GG,GG,GG
2088956,GGGGGGGGG,0.5,1,52.512866,G,G,G,G,G,G,...,G,G,GG,GG,GG,GG,GG,GG,GG,GG
2088957,GGGGGGGGG,0.1,2,46.070145,G,G,G,G,G,G,...,G,G,GG,GG,GG,GG,GG,GG,GG,GG
2088958,GGGGGGGGG,0.2,2,49.779454,G,G,G,G,G,G,...,G,G,GG,GG,GG,GG,GG,GG,GG,GG


Now we create the one-hot encoded columns similarly to how we did above.

In [31]:
df_bps_oh = df_bps.copy()
i = 0
while True:
    base = 'base_'
    # get out of the loop when we reach the last 'base_n' column
    try:
        df_bps[base + str(i + 1)]
    except KeyError:
        break
    # get new one-hot encoded columns
    target_col = base + str(i) + str(i + 1)
    new_cols = pd.get_dummies(df_bps[target_col], prefix = target_col)
    # concatenate them to the dataframe and drop the original columns
    df_bps_oh = pd.concat([df_bps_oh, new_cols], axis=1)
    df_bps_oh.drop([target_col, base + str(i)], axis=1, inplace=True)
    i += 1
df_bps_oh.drop(['seq', base + str(i)], axis=1, inplace=True)
df_bps_oh

,s_mol,c_umol,T_m,base_01_AA,base_01_AC,base_01_AG,base_01_AT,base_01_CA,base_01_CC,base_01_CG,...,base_78_G0,base_78_GA,base_78_GC,base_78_GG,base_78_GT,base_78_T0,base_78_TA,base_78_TC,base_78_TG,base_78_TT
0,0.1,1,-20.495855,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.2,1,-17.536560,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.5,1,-13.516521,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.1,2,-17.837395,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.2,2,-14.815124,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2088955,0.2,1,47.545073,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2088956,0.5,1,52.512866,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2088957,0.1,2,46.070145,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2088958,0.2,2,49.779454,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


Once the variables are encoded, we can load the data into dataloader objects:

In [32]:
# Cast the data into a dataset
y_bps = torch.Tensor(df_bps.T_m)
x_bps = torch.Tensor(df_bps_oh.drop('T_m', axis=1).values)

dataset_bps = TensorDataset(x_bps, y_bps)

train_length_bps = int(len(dataset_bps) * 0.9)
valid_length_bps = len(dataset_bps) - train_length_bps
train_set_bps, valid_set_bps = torch.utils.data.random_split(dataset_bps, [train_length_bps, valid_length_bps])

# Report split sizes
print(f'Training set has {len(train_set_bps)} instances')
print(f'Validation set has {len(valid_set_bps)} instances')

# Create the loaders
batch_size_bps = 64
training_loader_bps = DataLoader(train_set_bps, batch_size=batch_size_bps, shuffle=False, num_workers=2)
validation_loader_bps = DataLoader(valid_set_bps, batch_size=batch_size_bps, shuffle=False, num_workers=2)

Training set has 1880064 instances
Validation set has 208896 instances


The model structure, loss and optimizer are the same as in the previous case

In [33]:
model_bps = FC(x_bps.shape[1], 1, x_bps.shape[1] * 2)
#loss_fn_bps = nn.MSELoss()
loss_fn_bps = nn.L1Loss()
optimizer_bps = torch.optim.Adam(model_bps.parameters())

We rewrite a training function, `train_one_epoch_bps` just renaming some of the variable it uses from above.

In [34]:
def train_one_epoch_bps(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(training_loader_bps):
        # Every data instance is an input + label pair
        inputs, labels = data

        # Zero your gradients for every batch!
        optimizer_bps.zero_grad()

        # Make predictions for this batch
        outputs = model_bps(inputs)

        # Compute the loss and its gradients
        labels = torch.unsqueeze(labels, 1)

        loss = loss_fn_bps(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer_bps.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(training_loader_bps) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss

Finally, let's run the training loop on this model too

In [35]:
# Initializing in a separate cell so we can easily add more epochs to the same run
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))
epoch_number = 0

EPOCHS = 5

best_vloss = 1_000_000.

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model_bps.train(True)
    avg_loss = train_one_epoch_bps(epoch_number, writer)

    # We don't need gradients on to do reporting
    model_bps.train(False)

    running_vloss = 0.0
    for i, vdata in enumerate(validation_loader_bps):
        vinputs, vlabels = vdata
        voutputs = model_bps(vinputs)
        vlabels = torch.unsqueeze(vlabels, 1) # added to ensure labels and outputs are of the same size
        vloss = loss_fn_bps(voutputs, vlabels)
        running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    # Log the running loss averaged per batch
    # for both training and validation
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch_number + 1)
    writer.flush()

    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path_bps = 'model_bps_{}_{}'.format(timestamp, epoch_number)
        torch.save(model_bps.state_dict(), model_path_bps)

    epoch_number += 1

EPOCH 1:
  batch 1000 loss: 3.1822671583890916
  batch 2000 loss: 2.153823205471039
  batch 3000 loss: 2.0897410534620287
  batch 4000 loss: 2.0579217829704284
  batch 5000 loss: 2.028912232875824
  batch 6000 loss: 1.9860537939071656
  batch 7000 loss: 1.9585571776628494
  batch 8000 loss: 1.9517711392641068
  batch 9000 loss: 1.9217170761823654
  batch 10000 loss: 1.9066286067962646
  batch 11000 loss: 1.87270246386528
  batch 12000 loss: 1.8746917355060577
  batch 13000 loss: 1.8473758735656738
  batch 14000 loss: 1.8387831867933273
  batch 15000 loss: 1.7919547945261
  batch 16000 loss: 1.7881101115942002
  batch 17000 loss: 1.76903054356575
  batch 18000 loss: 1.7424039701223373
  batch 19000 loss: 1.7010922545194627
  batch 20000 loss: 1.6498824585676193
  batch 21000 loss: 1.6243142762184144
  batch 22000 loss: 1.5981925272941588
  batch 23000 loss: 1.576833158493042
  batch 24000 loss: 1.5528614037036896
  batch 25000 loss: 1.5299864151477813
  batch 26000 loss: 1.5119411797523

Now the MSE loss decreases all the way to `xxx` and the L1 loss gets to `xxx`. Let's see if we can get even lower than that.

In [285]:
inputs, labels = next(iter(training_loader))
model(inputs)

tensor([[26.9623],
        [23.6786],
        [21.2388],
        [25.3707],
        [16.7135],
        [35.6994],
        [17.4662],
        [20.6185],
        [20.8223],
        [40.8590],
        [28.3726],
        [31.9507],
        [28.1155],
        [26.7604],
        [45.5279],
        [28.2286],
        [28.4248],
        [39.4575],
        [35.6994],
        [45.3415],
        [26.1465],
        [35.6994],
        [20.9257],
        [33.5044],
        [16.7135],
        [28.4644],
        [18.7368],
        [26.2195],
        [20.7873],
        [39.5623],
        [31.9306],
        [35.6994],
        [16.7135],
        [19.2009],
        [20.6243],
        [22.1057],
        [36.8177],
        [18.7416],
        [28.3219],
        [31.0323],
        [28.1812],
        [30.9107],
        [16.7135],
        [32.6798],
        [27.5582],
        [35.7365],
        [45.0228],
        [16.7135],
        [32.2754],
        [21.8763],
        [35.6994],
        [30.7928],
        [35.

## 3. One-hot encoded bases fed to a recurring neural network and fully connected network
The DNA sequence can be expressed as a string, so it feels natural to try and tackle this problem with a recurring neural network. Let's give it a shot!

In [14]:
###########
# We can't reuse df_oh because that does not encode the possibiilty of having base 0 in the first few positions.
# This means that the one-hot encoded vectors don't have all the same dimensions.
df_oh_f = df.copy()
df_temp = df.copy()
df_temp.loc[df.shape[0]] = df_temp.shape[1] * [0]

i = 0
while True:
    target_col = 'base_' + str(i)
    # get out of the loop when we reach the last 'base_n' column
    try:
        df_temp[target_col]
    except KeyError:
        break
    new_cols = pd.get_dummies(df_temp[target_col], prefix = target_col)
    df_oh_f = pd.concat([df_oh_f, new_cols], axis=1)
    df_oh_f.drop(target_col, axis=1, inplace=True)
    i += 1
df_oh_f.drop('seq', axis=1, inplace=True)
df_oh_f.drop(df_oh_f.shape[0]-1,inplace=True)
print(df_oh_f.shape)
df_oh_f



(2088960, 48)


,s_mol,c_umol,T_m,base_0_0,base_0_A,base_0_C,base_0_G,base_0_T,base_1_0,base_1_A,...,base_7_0,base_7_A,base_7_C,base_7_G,base_7_T,base_8_0,base_8_A,base_8_C,base_8_G,base_8_T
0,0.1,1.0,-20.495855,0,1,0,0,0,0,1,...,1,0,0,0,0,1,0,0,0,0
1,0.2,1.0,-17.536560,0,1,0,0,0,0,1,...,1,0,0,0,0,1,0,0,0,0
2,0.5,1.0,-13.516521,0,1,0,0,0,0,1,...,1,0,0,0,0,1,0,0,0,0
3,0.1,2.0,-17.837395,0,1,0,0,0,0,1,...,1,0,0,0,0,1,0,0,0,0
4,0.2,2.0,-14.815124,0,1,0,0,0,0,1,...,1,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2088955,0.2,1.0,47.545073,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2088956,0.5,1.0,52.512866,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2088957,0.1,2.0,46.070145,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2088958,0.2,2.0,49.779454,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0


Like we did in the previous examples, we parse the dataset into `DataLoader` objects 

In [15]:
# Cast the data into a dataset
y_rnn = torch.Tensor(df_oh_f.T_m)
x_rnn = torch.Tensor(df_oh_f.drop('T_m', axis=1).values)


dataset = TensorDataset(x_rnn, y_rnn)

train_length = int(len(dataset) * 0.9)
valid_length = len(dataset) - train_length
train_set, valid_set = torch.utils.data.random_split(dataset, [train_length, valid_length])

# Report split sizes
print(f'Training set has {len(train_set)} instances')
print(f'Validation set has {len(valid_set)} instances')

# Create the loaders
batch_size = 16
training_loader_rnn = DataLoader(train_set, batch_size=batch_size, shuffle=False, num_workers=2)
validation_loader_rnn = DataLoader(valid_set, batch_size=batch_size, shuffle=False, num_workers=2)

Training set has 1880064 instances
Validation set has 208896 instances


We will reuse the dataloaders from point 1 above, so there is no need to define new dataloaders here.

On the other hand, we used a flattened one-hot encoding in both the models above, but in order to use the `nn.RNN` module we will need to store the one-hot encoded variables into a tensor with shape `(batch_size, seq_len, n_bases)`. The other two variables (duplex and salt concentrations) can be fed as they are to the fully-connected network. Let's write a function to convert the one-hot variables into a tensor, to be called from within the `Model`:

In [41]:
n_bases = 5 # 4 possible letters among A, T, C, G, and 0 for no base.
def get_conc_features(x):
    '''
    Given a batch of data x, return:
    1. a tensor of shape (batch_size, 2) containing the concentrations, and
    2. a tensor of shape (batch_size, seq_len, n_bases) containing the one-hot encoded sequence variables
    '''
    concs = x[:,:2]
    oh_flat = x[:, 2:]
    seq_len = df.shape[1] - 4# this uses the unprocessed dataframe read from file
    
    oh = torch.reshape(oh_flat, (x.shape[0],seq_len, n_bases))

    return concs, oh


Now we can define the model that we're going to use: a simple recurring neural network, whose output is fed to a fully connected network together with duplex and salt concentrations. We also define loss function and optimizer.

In [42]:
class RNNFC(nn.Module):
    def __init__(self, input_size, hidden_dim, seq_len):
        super(RNNFC, self).__init__()
        # Defining some parameters
        self.hidden_dim = hidden_dim
        n_conc = 2
        fc_hidden_dim = hidden_dim * seq_len + n_conc
        #Defining the layers
        # RNN Layer
        # using relu nonlinearity to prevent vanishing gradient problems
        self.rnn = nn.RNN(input_size, hidden_dim, batch_first=True, nonlinearity='relu')   
        # Fully connected layers, receiving the output of rnn and the two concentrations
        self.fc1 = nn.Linear(fc_hidden_dim, fc_hidden_dim)
        self.fc2 = nn.Linear(fc_hidden_dim, 1)
    def forward(self, x):
        batch_size = x.size(0)

        # Split the input tensor between concentrations and one-hot encoded variables
        concs, features = get_conc_features(x)

        # Passing features into the rnn
        # no need to pass a hidden state as it will be initialized to zero by default
        out, hidden = self.rnn(features)
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out.contiguous().view(batch_size, -1)
        
        # concatenating the outputs to the concentrations and feeding them to the
        # fully connected layers
        out = torch.cat((concs, out), axis=1)
        out = F.relu(self.fc1(out))
        out = self.fc2(out)
        
        return out

seq_len = df.shape[1] - 4
model_rnn = RNNFC(n_bases, 3, seq_len)
#loss_fn_rnn = nn.MSELoss()
loss_fn_rnn = nn.L1Loss()
optimizer_rnn = torch.optim.Adam(model_rnn.parameters())

We also rewrite the training function, `train_one_epoch_rnn`, similarly to how we did above.

In [43]:
def train_one_epoch_rnn(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(training_loader):
        # Every data instance is an input + label pair
        inputs, labels = data

        # Zero your gradients for every batch!
        optimizer_rnn.zero_grad()

        # Make predictions for this batch
        outputs = model_rnn(inputs)

        # Compute the loss and its gradients
        labels = torch.unsqueeze(labels, 1)
        
        loss = loss_fn_rnn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer_rnn.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(training_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss

Finally, let's run the training loop on this model too

In [44]:
# Initializing in a separate cell so we can easily add more epochs to the same run
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))
epoch_number = 0

EPOCHS = 5

best_vloss = 1_000_000.

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model_rnn.train(True)
    avg_loss = train_one_epoch_rnn(epoch_number, writer)

    # We don't need gradients on to do reporting
    model_rnn.train(False)

    running_vloss = 0.0
    for i, vdata in enumerate(validation_loader):
        vinputs, vlabels = vdata
        voutputs = model_rnn(vinputs)
        vlabels = torch.unsqueeze(vlabels, 1) # added to ensure labels and outputs are of the same size
        vloss = loss_fn_rnn(voutputs, vlabels)
        running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    # Log the running loss averaged per batch
    # for both training and validation
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch_number + 1)
    writer.flush()

    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path_rnn = 'model_rnn_{}_{}'.format(timestamp, epoch_number)
        torch.save(model_rnn.state_dict(), model_path_rnn)

    epoch_number += 1


EPOCH 1:
  batch 1000 loss: 7.492772699594497
  batch 2000 loss: 2.0802864763736726
  batch 3000 loss: 1.8112838703393936
  batch 4000 loss: 1.7793192170858383
  batch 5000 loss: 1.7654063835144043
  batch 6000 loss: 1.7470459654331207
  batch 7000 loss: 1.7492566611766815
  batch 8000 loss: 1.7193593571186065
  batch 9000 loss: 1.7160402928590774
  batch 10000 loss: 1.7246229212284088
  batch 11000 loss: 1.7057601486444474
  batch 12000 loss: 1.7130340725183486
  batch 13000 loss: 1.7112231035232544
  batch 14000 loss: 1.6990134575366973
  batch 15000 loss: 1.710004292488098
  batch 16000 loss: 1.7028482646942138
  batch 17000 loss: 1.714467954158783
  batch 18000 loss: 1.698732457280159
  batch 19000 loss: 1.6976447479724883
  batch 20000 loss: 1.6979927409887314
  batch 21000 loss: 1.701554960012436
  batch 22000 loss: 1.6780445975065232
  batch 23000 loss: 1.6922220222949982
  batch 24000 loss: 1.6896389483213425
  batch 25000 loss: 1.6833447102308274
  batch 26000 loss: 1.67575420

## 4. One-hot encoded bases fed to a recurring neural network and fully connected network

Finally, we try replacing the RNN layer with a LSTM layer.

To do so, we can keep the same loss and optimizer, and even reuse the `Model` object we used for the previous model: we just have to redefine its `rnn` layer to use a `LTSM` module.

In [22]:
model_lstm = RNNFC(n_bases, 3, seq_len)
model_lstm.rnn = nn.LSTM(n_bases, 3, batch_first=True)
#loss_fn_lstm = nn.MSELoss()
loss_fn_lstm = nn.L1Loss()
optimizer_lstm = torch.optim.Adam(model_lstm.parameters())

In [25]:
def train_one_epoch_lstm(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(training_loader_rnn):
        # Every data instance is an input + label pair
        inputs, labels = data

        # Zero your gradients for every batch!
        optimizer_lstm.zero_grad()

        # Make predictions for this batch
        outputs = model_lstm(inputs)

        # Compute the loss and its gradients
        labels = torch.unsqueeze(labels, 1)
        
        loss = loss_fn_lstm(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer_lstm.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(training_loader_rnn) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss

Finally, let's run the training loop on this model too

In [27]:
# Initializing in a separate cell so we can easily add more epochs to the same run
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))
epoch_number = 0

EPOCHS = 5

best_vloss = 1_000_000.

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model_lstm.train(True)
    avg_loss = train_one_epoch_lstm(epoch_number, writer)

    # We don't need gradients on to do reporting
    model_lstm.train(False)

    running_vloss = 0.0
    for i, vdata in enumerate(validation_loader_rnn):
        vinputs, vlabels = vdata
        voutputs = model_lstm(vinputs)
        vlabels = torch.unsqueeze(vlabels, 1) # added to ensure labels and outputs are of the same size
        vloss = loss_fn_lstm(voutputs, vlabels)
        running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    # Log the running loss averaged per batch
    # for both training and validation
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch_number + 1)
    writer.flush()

    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path_lstm = 'model_lstm_{}_{}'.format(timestamp, epoch_number)
        torch.save(model_lstm.state_dict(), model_path_lstm)

    epoch_number += 1



EPOCH 1:
  batch 1000 loss: 0.15579303295910357
  batch 2000 loss: 0.15416647762805225
  batch 3000 loss: 0.15281109505146742
  batch 4000 loss: 0.15363422441482544
  batch 5000 loss: 0.15695616906136275
  batch 6000 loss: 0.152926789291203
  batch 7000 loss: 0.152254264049232
  batch 8000 loss: 0.15166468101739883
  batch 9000 loss: 0.15027427373081445
  batch 10000 loss: 0.1522151111662388
  batch 11000 loss: 0.15276156885176898
  batch 12000 loss: 0.15270956530421972
  batch 13000 loss: 0.14585737206041813
  batch 14000 loss: 0.1491754691451788
  batch 15000 loss: 0.14894684515893458
  batch 16000 loss: 0.15191317104548216
  batch 17000 loss: 0.1492836202606559
  batch 18000 loss: 0.1482049672305584
  batch 19000 loss: 0.1518049224987626
  batch 20000 loss: 0.14886083249002696
  batch 21000 loss: 0.1440218227878213
  batch 22000 loss: 0.14998532424122096
  batch 23000 loss: 0.1474542141929269
  batch 24000 loss: 0.14499065185338258
  batch 25000 loss: 0.14775263911485673
  batch 260